# The SOLID Design Principles

### 1.Single Responsibility Principle
if you have a class that class should have its primary responsibility


In [ ]:
"""
SRP SOC
Anti Pattern GOD Object
everything into a single class
"""
class Journal:
    def __init__(self):
        self.entries = []
        self.count = 0
    def add_entry(self, text):
        self.count += 1
        self.entries.append(f"{self.count}: {text}")
    def save(self, filename): # you cannot check the directory 
        file = open(filename, 'w')
        file.write(str(self))
        file.close()
    def __str__(self):
        return '\n'.join(self.entries)
    
j = Journal()
j.add_entry("I fixed a bug")
j.add_entry("I added a new feature")
print(j)

In [5]:
"""
SRP SOC
"""
class Journal:
    def __init__(self):
        self.entries = []
        self.count = 0
    def add_entry(self, text):
        self.count += 1
        self.entries.append(f"{self.count}: {text}")
    def __str__(self):
        return '\n'.join(self.entries)
    
class PersistenceManager:
    @staticmethod
    def save_to_file(journal, filename):
        file = open(filename, 'w')
        file.write(str(journal))
        file.close()
        
j = Journal()
j.add_entry("I fixed a bug")
j.add_entry("I added a new feature")
print(j)
file = "/tmp/hi.txt"
PersistenceManager.save_to_file(j, file)

1: I fixed a bug
2: I added a new feature


### 2.Open-Closed Principle
OCP = Open for extension, closed for  modification

In [6]:
from enum import Enum

class Color(Enum):
    RED = 1
    GREEN = 2
    BLUE = 3
class Size(Enum):
    SMALL = 1
    MEDIUM = 2
    LARGE = 3
    
class Product:
    def __init__(self, name, color, size):
        self.name = name
        self.color = color
        self.size = size
        
class ProductFilter:
    def filter_by_color(self, products, color):
        for p in products:
            if p.color == color: yield p
    def filter_by_size(self, products, size):
        for p in products:
            if p.size == size: yield p
    def filter_by_size_and_color(self, products, size, color):
        for p in products:
            if p.color == color and p.size == size:
                yield p

In [7]:
# Specification EE Pattern

class Specification:
    def is_satisfied(self, item):
        pass

    # and operator makes life easier
    def __and__(self, other):
        return AndSpecification(self, other)


class Filter:
    def filter(self, items, spec):
        pass


class ColorSpecification(Specification):
    def __init__(self, color):
        self.color = color

    def is_satisfied(self, item):
        return item.color == self.color


class SizeSpecification(Specification):
    def __init__(self, size):
        self.size = size

    def is_satisfied(self, item):
        return item.size == self.size


# class AndSpecification(Specification):
#     def __init__(self, spec1, spec2):
#         self.spec2 = spec2
#         self.spec1 = spec1
#
#     def is_satisfied(self, item):
#         return self.spec1.is_satisfied(item) and \
#                self.spec2.is_satisfied(item)

class AndSpecification(Specification):
    def __init__(self, *args):
        self.args = args

    def is_satisfied(self, item):
        return all(map(
            lambda spec: spec.is_satisfied(item), self.args))


class BetterFilter(Filter):
    def filter(self, items, spec):
        for item in items:
            if spec.is_satisfied(item):
                yield item


apple = Product('Apple', Color.GREEN, Size.SMALL)
tree = Product('Tree', Color.GREEN, Size.LARGE)
house = Product('House', Color.BLUE, Size.LARGE)

products = [apple, tree, house]

pf = ProductFilter()
print('Green products (old):')
for p in pf.filter_by_color(products, Color.GREEN):
    print(f' - {p.name} is green')

# ^ BEFORE

# v AFTER
bf = BetterFilter()

print('Green products (new):')
green = ColorSpecification(Color.GREEN)
for p in bf.filter(products, green):
    print(f' - {p.name} is green')

print('Large products:')
large = SizeSpecification(Size.LARGE)
for p in bf.filter(products, large):
    print(f' - {p.name} is large')

print('Large blue items:')
# large_blue = AndSpecification(large, ColorSpecification(Color.BLUE))
large_blue = large & ColorSpecification(Color.BLUE)
for p in bf.filter(products, large_blue):
    print(f' - {p.name} is large and blue')

### 3.Liskov Substitution Principle
if you have some interface that takes some sort of base class
you should be able to  stick a derived class in there and everything
should work

In [8]:
from abc import ABC, abstractmethod

class Notifier(ABC):
    def trigger(name, body, email):
        pass
        
class CostumerNotifier(Notifier):
    def trigger(self, name, body, email):
        print(f"{name} : {body} to {email}")
        
class DeveloperNotifier(Notifier):
    def trigger(self, name, body, email):
        print(f"{name} : {body} to {email}")
        
DeveloperNotifier().trigger(name="Ahmad", body="low disk space", email="091254849464")

Ahmad : low disk space to 091254849464


In [9]:
from abc import ABC, abstractmethod

class Notifier(ABC):
    def trigger(name, body):
        pass
        
class CostumerNotifier(Notifier):
    def __init__(self, email):
        self.email = email
    def trigger(self, name, body):
        print(f"{name} : {body} to {self.email}")
        
class DeveloperNotifier(Notifier):
    def __init__(self, phone):
        self.phone = phone
    def trigger(self, name, body):
        print(f"{name} : {body} to {self.phone}")
        
DeveloperNotifier(phone="091254849464").trigger(name="Ahmad", body="low disk space")

Ahmad : low disk space to 091254849464


In [1]:
"""
LSP
"""
class Rectangle:
    def __init__(self, width, height):
        self._height = height
        self._width = width

    @property
    def area(self):
        return self._width * self._height

    def __str__(self):
        return f'Width: {self.width}, height: {self.height}'

    @property
    def width(self):
        return self._width

    @width.setter
    def width(self, value):
        self._width = value

    @property
    def height(self):
        return self._height

    @height.setter
    def height(self, value):
        self._height = value


class Square(Rectangle):
    def __init__(self, size):
        Rectangle.__init__(self, size, size)

    @Rectangle.width.setter
    def width(self, value):
        _width = _height = value

    @Rectangle.height.setter
    def height(self, value):
        _width = _height = value


def use_it(rc):
    w = rc.width
    rc.height = 10  # unpleasant side effect
    expected = int(w * 10)
    print(f'Expected an area of {expected}, got {rc.area}')


rc = Rectangle(2, 3)
use_it(rc)

sq = Square(5)
use_it(sq)


Expected an area of 20, got 20
Expected an area of 50, got 25


### 4.Interface Segregation Principle
you don't really want to stick to many elements too many mehtods
for example into an interface

In [ ]:
from abc import abstractmethod


class Machine:
    def print(self, document):
        raise NotImplementedError()

    def fax(self, document):
        raise NotImplementedError()

    def scan(self, document):
        raise NotImplementedError()


# ok if you need a multifunction device
class MultiFunctionPrinter(Machine):
    def print(self, document):
        pass

    def fax(self, document):
        pass

    def scan(self, document):
        pass


class OldFashionedPrinter(Machine):
    def print(self, document):
        # ok - print stuff
        pass

    def fax(self, document):
        pass  # do-nothing

    def scan(self, document):
        """Not supported!"""
        raise NotImplementedError('Printer cannot scan!')


class Printer:
    @abstractmethod
    def print(self, document): pass


class Scanner:
    @abstractmethod
    def scan(self, document): pass


# same for Fax, etc.

class MyPrinter(Printer):
    def print(self, document):
        print(document)


class Photocopier(Printer, Scanner):
    def print(self, document):
        print(document)

    def scan(self, document):
        pass  # something meaningful


class MultiFunctionDevice(Printer, Scanner):  # , Fax, etc
    @abstractmethod
    def print(self, document):
        pass

    @abstractmethod
    def scan(self, document):
        pass


class MultiFunctionMachine(MultiFunctionDevice):
    def __init__(self, printer, scanner):
        self.printer = printer
        self.scanner = scanner

    def print(self, document):
        self.printer.print(document)

    def scan(self, document):
        self.scanner.scan(document)


printer = OldFashionedPrinter()
printer.fax(123)  # nothing happens
printer.scan(123)  # oops!


### 5.Dependency Inversion Principle 
high level modules should not depend directly onn low level modules
Instead they shoould depend on abstractions

In [2]:
from abc import abstractmethod
from enum import Enum


class Relationship(Enum):
    PARENT = 0
    CHILD = 1
    SIBLING = 2


class Person:
    def __init__(self, name):
        self.name = name


class RelationshipBrowser:
    @abstractmethod
    def find_all_children_of(self, name): pass


class Relationships(RelationshipBrowser):  # low-level
    relations = []

    def add_parent_and_child(self, parent, child):
        self.relations.append((parent, Relationship.PARENT, child))
        self.relations.append((child, Relationship.PARENT, parent))
            
    def find_all_children_of(self, name):
        for r in self.relations:
            if r[0].name == name and r[1] == Relationship.PARENT:
                yield r[2].name


class Research:
    # dependency on a low-level module directly
    # bad because strongly dependent on e.g. storage type

    # def __init__(self, relationships):
    #     # high-level: find all of john's children
    #     relations = relationships.relations
    #     for r in relations:
    #         if r[0].name == 'John' and r[1] == Relationship.PARENT:
    #             print(f'John has a child called {r[2].name}.')

    def __init__(self, browser):
        ...


# KISS Principles ( Keep it simple, stupid)
most systems woork best if they are simple rather than made complicated.
simplicity should be a key goal in design, and unnecessary complexity should be avoided.

In [4]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


# DRY (Don't repeat yourself)
code should not be duplicated. 
instead of duplicating lines, find an algorithm that uses iteration

# SoC (Separation of concerns)
a good example of Soc is MVC (Model - View - Controller)